In [20]:
import numpy as np
from typing import List, Dict, Union, Tuple
from tenpy.networks.site import SpinSite
from tenpy.linalg import np_conserved as npc
from tenpy.linalg.np_conserved import Array
from tenpy.networks.mpo import MPO
from tenpy.models.model import MPOModel
from tenpy.models.lattice import Chain
from tenpy.networks.mps import MPS  

In [5]:
def random_L(d: int, Dl: int, Dr: int, seed=None):
    """
    Tensore L_k di shape (d, Dl, Dr).
    Ogni matrice L[x] ha un numero casuale di righe nulle (da 0 a Dl).
    Su ogni riga non nulla c’è un solo '1' in colonna casuale.
    """
    rng = np.random.default_rng(seed)
    L = np.zeros((d, Dl, Dr), dtype=np.uint8)

    for x in range(d):
        # scegli quante righe saranno attive
        n_active = rng.integers(0, Dl + 1)
        # scegli quali righe sono attive
        active_rows = rng.choice(Dl, size=n_active, replace=False)
        # assegna un '1' casuale per ciascuna riga attiva
        for i in active_rows:
            j = rng.integers(Dr)
            L[x, i, j] = 1
    return L

def random_R(d: int, Dl: int, Dr: int, seed=None):
    """
    Tensore R_k di shape (d, Dl, Dr).
    Ogni matrice R[x] ha un numero casuale di colonne nulle (da 0 a Dr).
    Su ogni colonna non nulla c’è un solo '1' in riga casuale.
    """
    rng = np.random.default_rng(seed)
    R = np.zeros((d, Dl, Dr), dtype=np.uint8)

    for x in range(d):
        # scegli quante colonne saranno attive
        n_active = rng.integers(0, Dr + 1)
        # scegli quali colonne sono attive
        active_cols = rng.choice(Dr, size=n_active, replace=False)
        # assegna un '1' casuale per ciascuna colonna attiva
        for j in active_cols:
            i = rng.integers(Dl)
            R[x, i, j] = 1
    
    return R

def random_A1(d: int, D: int, seed=None):
    """Primo sito: shape (d, 1, D). Binario libero."""
    rng = np.random.default_rng(seed)
    return rng.integers(0, 2, size=(d, 1, D), dtype=np.uint8)

def wrap_site_tensor(T: np.ndarray):
    """(d, Dl, Dr) -> npc.Array labels ['vL','p','vR'] shape (Dl, d, Dr)"""
    Dl = T.shape[1]
    Ai = np.transpose(T, (1, 0, 2))  # (Dl, d, Dr)
    return npc.Array.from_ndarray_trivial(Ai, labels=['vL', 'p', 'vR'])

def tenpy_sites_and_svs(d: int, right_dims):
    N = len(right_dims)
    S = (d - 1) / 2.0
    site = SpinSite(S=S, conserve=None)
    lattice = Chain(L=N, site=site)
    sites = lattice.mps_sites()
    svs = [np.ones(1, dtype=float)]
    svs += [np.ones(right_dims[i], dtype=float) for i in range(N - 1)]
    svs += [np.ones(1, dtype=float)]
    return sites, svs

In [22]:
def site_tensor_from_Bi(Bi: dict, d: int):
    # Costruisce T[x] = Bi[x] con fallback a zeri. T shape: (d, Dl, Dr).
    shapes = {np.shape(M) for M in Bi.values()}
    if len(shapes) != 1:
        raise ValueError(f"shape incoerenti {shapes}")
    Dl, Dr = next(iter(shapes))
    dtype = np.result_type(*[np.asarray(Bi.get(x, 0)).dtype for x in range(d)])
    T = np.zeros((d, Dl, Dr), dtype=dtype)
    for x in range(d):
        Mx = np.asarray(Bi.get(x, np.zeros((Dl, Dr), dtype=dtype)), dtype=dtype)
        if Mx.shape != (Dl, Dr):
            raise ValueError(f"B[{x}] ha shape {Mx.shape}, atteso {(Dl,Dr)}")
        T[x] = Mx
    return T, Dl, Dr

def build_mps(B_list, d: int):
    N = len(B_list)
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere intero positivo.")

    tensors = []
    right_dims = []

    for i, Bi in enumerate(B_list):
        T, Dl, Dr = site_tensor_from_Bi(Bi, d)
        if i == 0 and Dl != 1:
            raise ValueError(f"sito {i}: Dl deve essere 1, trovato {Dl}")
        if i == N - 1 and Dr != 1:
            raise ValueError(f"sito {i}: Dr deve essere 1, trovato {Dr}")
        tensors.append(T)
        right_dims.append(Dr)

    A = [wrap_site_tensor(T) for T in tensors]
    sites, svs = tenpy_sites_and_svs(d, right_dims)
    return MPS(sites, A, svs, bc='finite', form='A')

In [7]:
def build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu):
    pmid = 1 - pd - pu
    if not (0 <= pd <= 1 and 0 <= pu <= 1 and pd + pu <= 1):
        raise ValueError("Probabilità non valide: servono pd, pu >=0 e pd+pu <= 1")

    B_list = []

    # Sito 1: (1,2)
    B1 = {
        0: np.array([[ (S0/(N+1))*d_op*pd,  (S0/(N+1))*d_op*pd - K*pd ]]),
        1: np.array([[ (S0/(N+1))*m_op*pmid, ((S0/(N+1))*m_op - K)*pmid ]]),
        2: np.array([[ (S0/(N+1))*u_op*pu,  (S0/(N+1))*u_op*pu - K*pu ]]),
    }
    B_list.append(B1)

    # Siti 2..N-1: (2,2)  ← QUI MANCANO NEL TUO FILE
    for i in range(2, N):
        Bi = {
            0: np.array([[ d_op*pd,  (S0/(N+1))*d_op*pd - K*pd ],
                         [     pd,                               1 ]]),
            1: np.array([[ m_op*pmid, ((S0/(N+1))*m_op - K)*pmid ],
                         [      pmid,                               1 ]]),
            2: np.array([[ u_op*pu,  (S0/(N+1))*u_op*pu - K*pu ],
                         [    pu,                                 1 ]]),
        }
        B_list.append(Bi)

    # Sito N: (2,1)
    BN = {
        0: np.array([[ d_op*pd ],
                     [     pd ]],),
        1: np.array([[ m_op*pmid ],
                     [      pmid ]]),
        2: np.array([[ u_op*pu ],
                     [     pu ]]),
    }
    B_list.append(BN)
    return B_list

In [8]:
def build_mps_AR(d: int, N: int, D: int, seed=None):
    """
    Crea b(x) con bond dimensione D:
      A1: (d, 1, D)
      R2..R_{N-1}: (d, D, D)
      RN: (d, D, 1)
    """
    if N < 2:
        raise ValueError("N deve essere >= 2 per avere A1 e almeno un R.")

    A1 = random_A1(d, D, seed=None)
    Rs = []
    for k in range(2, N):
        Rs.append(random_R(d, Dl=D, Dr=D, seed=None))
    RN = random_R(d, Dl=D, Dr=1, seed=None)
    tensors = [A1] + Rs + [RN]

    # wrap TenPy
    A = [wrap_site_tensor(T) for T in tensors]
    right_dims = [T.shape[2] for T in tensors]  # Dr per ciascun sito
    sites, svs = tenpy_sites_and_svs(d, right_dims)
    return MPS(sites, A, svs, bc='finite', form='A')


In [9]:
S0, K, N = 1.0, 0.2, 4
d_op, m_op, u_op = 0.9, 1.0, 3
pd, pu = 0.2, 0.4   

B_list = build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu)

psi = build_mps(B_list, d=3)

b = build_mps_AR(d=3, N=len(B_list), D=10)

val = b.overlap(psi)
print("sum_x psi(x)b(x) =", val)

sum_x psi(x)b(x) = 0.12453715726622543


In [10]:
def contract_left(tensor, i, N):
    A = [tensor.get_B(k) for k in range(i-1)]   # each: (Dl, d, Dr)
    #print(len(A))

    if i == 1:
        return None   # oppure semplicemente "return" per terminare senza valore

    if i > N:
        raise ValueError(f"Indice i={i} maggiore del numero di siti N={N}.")
    
    if i <= 0:
        raise ValueError(f"Indice i={i} minore o uguale a 0.")


    # ncon index bookkeeping:
    # open legs use negative integers; positive integers are contracted
    # A0: (-1, -2,  1)
    # A1: ( 1, -3,  2)
    # A2: ( 2, -4,  3)
    # ...
    # A_{i-2}: (i-2, -(i), -(i+1))  # the final right bond stays open

    con_tensors = []
    con_indices = []
    if len(A) == 1:
        con_indices.append([-1, -2, -3])
        con_tensors.append(A[0])
    else:
        for k, Ak in enumerate(A):
            if k == 0:                                  # first site
                con_tensors.append(Ak)
                con_indices.append([-1, -2, 1])
            elif k < len(A) - 1:                        # middle sites
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), k+1])
            else:                                       # last of the block
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), -(k+3)])
    #print(con_indices)

    # result has open legs: [-1 (left=1), -2..-(i) (physicals), -(i+1) (right bond Di-1)]
    T = ncon(con_tensors, con_indices)
    # T.shape == (1, d, d, ..., d, D_{i-1})   # (i-1) copies of d

    return T

def contract_right(tensor, i, N):
    A = [tensor.get_B(k) for k in range(i,N)]   # each: (Dl, d, Dr)
    #print(len(A))

    if i == N:
        return None   # oppure semplicemente "return" per terminare senza valore

    if i > N:
        raise ValueError(f"Indice i={i} maggiore del numero di siti N={N}.")
    
    if i <= 0:
        raise ValueError(f"Indice i={i} minore o uguale a 0.")

    # ncon index bookkeeping:
    # open legs use negative integers; positive integers are contracted
    # Ai: (-1, -2,  1)
    # Ai+1: ( 1, -3,  2)
    # Ai+2: ( 2, -4,  3)
    # ...
    # AN-1: (i-2, -(i), -(i+1))  # the final right bond stays open

    con_tensors = []
    con_indices = []
    if len(A) == 1:
        con_indices.append([-1, -2, -3])
        con_tensors.append(A[0])
    else:
        for k, Ak in enumerate(A):
            if k == 0:                                  # first site
                con_tensors.append(Ak)
                con_indices.append([-1, -2, 1])
            elif k < len(A) - 1:                        # middle sites
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), k+1])
            else:                                       # last of the block
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), -(k+3)])
    #print(con_indices)

    # result has open legs: [-1 (left=1), -2..-(i) (physicals), -(i+1) (right bond Di-1)]
    T = ncon(con_tensors, con_indices)
    # T.shape == (1, d, d, ..., d, D_{i-1})   # (i-1) copies of d
    return T

UL = contract_left(tensor=psi,i=3,N=len(B_list))
UR = contract_right(tensor=psi,i=3,N=len(B_list))


In [11]:
def sign_tensorial_derivative(psi, b, site, N = len(B_list)):
    
    UL = contract_left(tensor=psi,i=site,N=len(B_list))
    UR = contract_right(tensor=psi,i=site,N=len(B_list))
    BL = contract_left(tensor=b,i=site,N=len(B_list))
    BR = contract_right(tensor=b,i=site,N=len(B_list))

    if UL is not None and len(UL.shape) != len(BL.shape):
        raise ValueError(f"Dimensione di UL diversa da BL")
    if UR is not None and len(UR.shape) != len(BR.shape):
        raise ValueError(f"Dimensione di UR diversa da BR")

    if UL is not None:
        left_tensors = [UL,BL]
        left_links = [
            [-1] + [k for k in range(1,site)] + [-2], 
        [-3] + [k for k in range(1,site)] + [-4] 
        ]
        print('left contraction links',left_links)
        left_contraction = ncon(left_tensors,left_links) 
    ## Il risultato è un tensore di dimensione (1,D_i;1,2)

    if UR is not None:
        right_tensors = [UR,BR]
        right_links = [
            [-1] + [k for k in range(1,N-site+1)] + [-2], 
        [-3] + [k for k in range(1,N-site+1)] + [-4] 
        ]
        print('right contraction links',right_links)
        right_contraction = ncon(right_tensors,right_links) 
    ## Il risultato è un tensore di dimensione (D_i+1,1;2,1)

    final_tensors = [left_contraction, b.get_B(site-1),right_contraction]
    final_links = [
        [-1] + [-2] + [-3] + [1],
        [1] + [-4] + [2],
        [-5] + [-6] + [2] + [-7]
    ]
    final_contraction = ncon(final_tensors,final_links)
    print(final_links)
    print(final_contraction.shape)

    return np.squeeze(final_contraction)  


In [28]:
sign_tensorial_derivative(psi=psi,b=b, site=2)

left contraction links [[-1, 1, -2], [-3, 1, -4]]
right contraction links [[-1, 1, 2, 3, 4, 5, 6, -2], [-3, 1, 2, 3, 4, 5, 6, -4]]
[[-1, -2, -3, 1], [1, -4, 2], [-5, -6, 2, -7]]
(1, 10, 1, 3, 10, 1, 1)


<npc.Array shape=(10, 3, 10) labels=[None, 'p', None]>